In [1]:
import math
from random import choice
from torch import empty

from generate_data import generate_data
from Modules import Sequential, Linear, ReLU, Tanh, Sigmoid, LossMSE

In [2]:
N = 1000

train_input, train_target, test_input, test_target = generate_data(1000, True)

print(train_input.shape)
print(train_input.std())
print(train_input.mean())

torch.Size([1000, 2])
tensor(1.0000)
tensor(-1.3313e-07)


In [3]:
r = ReLU()
print(r.forward(train_input))
print(r.backward(test_input))
print(r.param())

tensor([[0.9496, -0.0000],
        [1.6129, 1.5926],
        [-0.0000, -0.0000],
        ...,
        [-0.0000, 1.5042],
        [-0.0000, -0.0000],
        [1.5637, -0.0000]])
tensor([[ 1.6827, -0.0000],
        [ 0.7218, -1.5687],
        [ 0.0000, -0.0000],
        ...,
        [ 0.0000,  0.6197],
        [-0.0000, -0.0000],
        [-0.3193,  0.0000]])
ReLU has no parameters
[]


In [4]:
t = Tanh()
print(t.forward(train_input))
print(t.backward(test_input))
print(t.param())

tensor([[ 0.7396, -0.5365],
        [ 0.9236,  0.9205],
        [-0.7399, -0.8757],
        ...,
        [-0.8640,  0.9059],
        [-0.7925, -0.3524],
        [ 0.9160, -0.4704]])
tensor([[ 0.7622, -1.0633],
        [ 0.1061, -0.2394],
        [ 0.3852, -0.2159],
        ...,
        [ 0.0949,  0.1111],
        [-0.2550, -0.4211],
        [-0.0514,  0.8304]])
Tanh has no parameters
[]


In [5]:
s = Sigmoid()
print(s.forward(train_input))
print(s.backward(test_input))
print(s.param())

tensor([[0.7210, 0.3545],
        [0.8338, 0.8310],
        [0.2788, 0.2047],
        ...,
        [0.2127, 0.8182],
        [0.2539, 0.4090],
        [0.8269, 0.3751]])
tensor([[ 0.3385, -0.3417],
        [ 0.1000, -0.2203],
        [ 0.1711, -0.1508],
        ...,
        [ 0.0627,  0.0922],
        [-0.1299, -0.1162],
        [-0.0457,  0.2499]])
Sigmoid has no parameters
[]


In [6]:
l = Linear(2, 3)

print(l.forward(empty(2).normal_()))
print(l.backward(empty(3).normal_()))
l.param()

tensor([ 4.0844e-01,  2.6811e+00, -1.0228e+11])
tensor([1.6715, 0.0437])


[(tensor([[ 0.6341,  0.1745],
          [ 2.4289,  2.1635],
          [-1.5185,  1.2460]]), tensor([[ 0.9817,  1.6720],
          [-0.0148, -0.0252],
          [-0.0964, -0.1642]])),
 (tensor([ 0.0000e+00,  0.0000e+00, -1.0228e+11]),
  tensor([ 2.2386, -0.0338, -0.2199]))]

In [3]:
l1 = Linear(4,6)
r1 = ReLU()
l2 = Linear(6, 8)
t2 = Tanh()

S = Sequential(l1, r1, l2, t2)
print(S.forward(empty(4).normal_()))
print(S.backward(empty(8).normal_()))
print(S.param())
S.name

tensor([-1., -1.,  1., -1., -1., -1.,  1., -1.])
tensor([0., 0., 0., 0.])
ReLU has no parameters
Tanh has no parameters
[[(tensor([[-1.2932,  1.7220,  1.3881, -0.7056],
        [-0.9080,  0.0234,  0.2244, -0.1977],
        [-0.9730,  0.3552,  1.1050,  0.0940],
        [ 0.5398,  0.6748,  1.1153, -1.4859],
        [ 1.7217, -1.2040, -0.0773, -0.3395],
        [-1.0466, -0.3869, -1.0521,  0.3376]]), tensor([[-0., 0., 0., 0.],
        [-0., 0., 0., 0.],
        [-0., 0., 0., 0.],
        [-0., 0., 0., 0.],
        [-0., 0., 0., 0.],
        [-0., 0., 0., 0.]])), (tensor([6.0022e+31, 4.2964e+24, 7.5555e+31, 1.1703e-19, 1.5637e-01, 1.1824e+22]), tensor([0., 0., 0., 0., 0., 0.]))], None, [(tensor([[-1.3878, -0.8782,  0.9062, -0.5198, -0.2773, -0.2306],
        [ 0.6775,  0.2731, -0.9427,  1.3577,  1.0647, -0.5824],
        [-0.5448, -0.3833,  0.7282, -0.2341,  2.1450,  1.1468],
        [-0.0872,  0.3840, -0.3850,  0.1755, -0.5099,  0.7178],
        [-1.0055,  1.7942, -1.1874,  0.6522, -0.321

'Linear(4x6) => ReLU => Linear(6x8) => Tanh'

In [5]:
def one_hot_encode(target):
    ohe_target = empty(target.shape[0], 2)
    ohe_target[:, 0] = train_target == 0
    ohe_target[:, 1] = train_target == 1
    return ohe_target

In [4]:
def train_model_SGD(model, train_input, train_target, nb_epoch = 10000):
    nb_train_samples = train_input.shape[0]
    ohe_train_target = one_hot_encode(train_target)
    mse = LossMSE()
    eta = 1e-1 / nb_train_samples
    for epoch in range(nb_epoch):
        n = choice(range(nb_train_samples))
        # Run forward pass
        output = model.forward(train_input[n])

        # Compute loss
        loss = mse.compute_loss(output, ohe_train_target[n])

        # Run back propagation
        model.backward(mse.backward())
        
        model.update_param(eta)

        # Log the loss
        if (epoch % 100 == 0):
            print(f"Epoch : {epoch}, Loss : {loss}")
    

In [6]:
l1 = Linear(2,25)
a1 = ReLU()
l2 = Linear(25, 25)
a2 = Tanh()
l3 = Linear(25, 25)
a3 = Sigmoid()
l4 = Linear(25, 2)

model = Sequential(l1, a1, l2, a2, l3, a3, l4)

In [7]:
train_model_SGD(model, train_input, train_target)

Epoch : 0, Loss : 3.0267820358276367
Epoch : 100, Loss : 18.4509334564209
Epoch : 200, Loss : 15.403722763061523
Epoch : 300, Loss : 12.540345191955566
Epoch : 400, Loss : 0.5148259401321411
Epoch : 500, Loss : 3.6549530029296875
Epoch : 600, Loss : 5.363900661468506
Epoch : 700, Loss : 0.5512146353721619
Epoch : 800, Loss : 2.7910304069519043
Epoch : 900, Loss : 0.07498374581336975
Epoch : 1000, Loss : 0.22721083462238312
Epoch : 1100, Loss : 0.8243176341056824
Epoch : 1200, Loss : 0.8627876043319702
Epoch : 1300, Loss : 3.4397644996643066
Epoch : 1400, Loss : 0.03304024785757065
Epoch : 1500, Loss : 0.9418724775314331
Epoch : 1600, Loss : 1.1240551471710205
Epoch : 1700, Loss : 1.6532535552978516
Epoch : 1800, Loss : 0.9338143467903137
Epoch : 1900, Loss : 0.38969096541404724
Epoch : 2000, Loss : 0.7475508451461792
Epoch : 2100, Loss : 3.6028199195861816
Epoch : 2200, Loss : 1.0302551984786987
Epoch : 2300, Loss : 0.3700354993343353
Epoch : 2400, Loss : 0.39728009700775146
Epoch : 25

In [53]:
def accuracy(model, input, target):
    output = empty(input.shape[0], 2)
    for n in range(output.shape[0]):
        output[n] = model.forward(train_input[n])
    return (output.argmax(1) == target).sum() / float(output.shape[0])

In [56]:
accuracy(model, train_input, train_target)

tensor(0.9680)

In [55]:
accuracy(model, test_input, test_target)

tensor(0.5770)